In [5]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt
from tip_finding.tip_finding import tip_finder_decimation

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load = pd.read_csv("agents/Data/endpoints_gt4.csv")
endpoints_dict = {}

In [7]:
load.head()

,neuron,ng_link,seg_id,pink_pts,num_endpoints,endpoints,comments,detailed_comments
0,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"((402584, 228856, 23991), (402985, 229235, 235...",NaN,NaN
1,NaN,NaN,864691135247440303,"(401258, 224832, 24029)",3,"((401612, 224623, 23991), (405257, 226318, 236...",NaN,NaN
2,NaN,NaN,864691134794123793,"(401314, 228366, 24424)",2,"((401242, 228382, 24444), (400982, 228457, 245...",NaN,NaN
3,NaN,NaN,864691134959738482,"(400199, 220721, 24029)",7,"((401289, 218721, 23991), (399895, 216533, 235...",NaN,NaN
4,NaN,NaN,864691135314714227,"(397870, 232292, 24004)",2,"((397867, 232230, 23999), (397854, 232252, 239...",NaN,NaN


In [8]:
#creating series objects for the desired collumns 
segIDs = load['seg_id']
Endpoints = load['endpoints']



In [9]:
for idx, i in enumerate(load['seg_id']):
    #if idx==10:
        #break
    try:
        t1, skel, mesh_obj = tip_finder_decimation(str(i))
        endpoints_dict[i] = t1
    except:
        print(f"\n\nSeg {i} returned error on get. Skipping.\n")
        pass

load['endpoints'] = load['endpoints'].apply(lambda x: list(ast.literal_eval(x)))
load['proposed_endpoints'] = load.seg_id.map(endpoints_dict)

07/22/2022 03:49:57 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 3996/3996 [00:00<00:00, 558811.68it/s]
07/22/2022 03:50:02 PM WARNING: face_normals incorrect shape, ignoring! 


  0%|          | 0/10649 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/meshparty/skeletonize.py:613: RuntimeWarning: invalid value encountered in multiply
  target = np.nanargmax(root_ds * valid)
100%|██████████| 10649/10649 [00:00<00:00, 444876.38it/s]
07/22/2022 03:50:04 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 1140/1140 [00:00<00:00, 405831.49it/s]
07/22/2022 03:50:06 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 2596/2596 [00:00<00:00, 527130.77it/s]
07/22/2022 03:50:07 PM WARNING: face_normals incorrect shape, ignoring! 


...

100%|██████████| 13258/13258 [00:00<00:00, 548284.22it/s]
07/22/2022 03:57:59 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 9618/9618 [00:00<00:00, 718498.48it/s]
07/22/2022 03:58:04 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 38734/38734 [00:00<00:00, 194500.49it/s]
07/22/2022 03:58:09 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 811/811 [00:00<00:00, 277816.12it/s]
07/22/2022 03:58:10 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 9313/9313 [00:00<00:00, 438814.97it/s]


In [11]:
#correctly  have 2 collumns for the proposed vs gt endpoints
load.head(15)

,neuron,ng_link,seg_id,pink_pts,num_endpoints,endpoints,comments,detailed_comments,proposed_endpoints
0,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"[(402584, 228856, 23991), (402985, 229235, 235...",NaN,NaN,"[(402567, 228758, 23991), (402990, 229222, 235..."
1,NaN,NaN,864691135247440303,"(401258, 224832, 24029)",3,"[(401612, 224623, 23991), (405257, 226318, 236...",NaN,NaN,"[(401467, 224644, 23990), (405021, 226754, 236..."
2,NaN,NaN,864691134794123793,"(401314, 228366, 24424)",2,"[(401242, 228382, 24444), (400982, 228457, 245...",NaN,NaN,"[(400269, 228563, 24514), (400319, 228912, 245..."
3,NaN,NaN,864691134959738482,"(400199, 220721, 24029)",7,"[(401289, 218721, 23991), (399895, 216533, 235...",NaN,NaN,"[(400448, 220975, 24009), (401240, 217916, 240..."
4,NaN,NaN,864691135314714227,"(397870, 232292, 24004)",2,"[(397867, 232230, 23999), (397854, 232252, 239...",NaN,NaN,"[(397815, 232393, 23996), (397931, 232013, 240..."
5,NaN,NaN,864691135552841025,"(403272, 227529, 24394)",2,"[(403332, 227545, 24398), (403566, 227614, 244...",NaN,NaN,"[(403094, 227479, 24424), (403709, 227788, 244..."
6,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135516937065,"(77233, 113188, 20454)",2,"[(77262, 113088, 20442), (79178, 109116, 20402)]",NaN,NaN,"[(77206, 113116, 20440), (79748, 109356, 20411)]"
7,NaN,NaN,864691134406233920,"(100382, 143836, 21642)",2,"[(100475, 143982, 21654), (100583, 144194, 217...",NaN,NaN,"[(100780, 144830, 21653), (100979, 144058, 216..."
8,NaN,NaN,864691135516937065,"(77233, 113188, 20454)",2,"[(77262, 113088, 20442), (79178, 109116, 20402)]",d,same point as previous endpoint,"[(77206, 113116, 20440), (79748, 109356, 20411)]"
9,NaN,NaN,864691136080776044,"(80699, 146603, 20732)",5,"[(80890, 147158, 20708), (81592, 148549, 20453...",mm,NaN,"[(80210, 147262, 20629), (80439, 146968, 20684..."


In [12]:

def find_euclidian_distance(proposed_endpoint, gt_endponts_array):
    smallest_distance = 2**30
    for endpoint in gt_endponts_array:
        difference = proposed_endpoint - endpoint
        distance = np.sqrt(np.sum(np.square(difference)))
        if(distance < smallest_distance):
            smallest_distance = distance 
    return smallest_distance 
            
    #could add endpoint to dict
    
     


In [2]:
def get_recall(num_correct, array_endpoints): 
    recall = num_correct / len(array_endpoints)
    return recall * 100

In [13]:
def get_precision(num_correct, array_proposed): 
    precision = num_correct / len(array_proposed)
    return precision * 100

In [14]:
#TEST 1: invalidation_d at default: 12000, WITH Humfrey_Smoothing: 50 iterations, Decimation_Factor = 0.50

#should track total amount of correct gt recovered 

for index, row in load.iterrows():
    #reset num correct
    num_correct = 0
    threshold = 500
    endpoint_array = np.array(row["endpoints"])
    proposed_endpoints_array = np.array(row["proposed_endpoints"])
    print(f"Seg id: %i" %(row["seg_id"]))
    print("Endpoint Array:")
    print(endpoint_array)
    #print(len(endpoint_array))
    print()
    print("Proposed Endpoint Array:")
    print(proposed_endpoints_array)
    print()
    #print(load["Endpoints"])

    #skip sharply angled/false merged
    if row["comments"] == 'a':
        print("skipping")
        print()
        continue

    #skip small seg ids
    if row["comments"] == 's':
        print("skipping")
        print()
        continue 
   
    #skip any seg ids with false merges: 
    if row["comments"] in ('m', "mm", "e"):
        print("skipping")
        print()
        continue

    #learn better pandas filtered rather than if else statements 

    #maybe skip v-shaped segments

    #add in a label for ground truth 
    
    #if both say no endpoints, then it's correct
    if (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) == 0):
        print("correct, no endpoints")
        print("\nNEXT POINTS")
        continue
    #if we propose no endpoints but there are endpoints, it's wrong
    elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) > 0):
        print("no proposed endpoints found, but ground truth endpoints exist")
        print("\nNEXT POINTS")
        continue 

    #if we propose endpoints but there are none, it's wrong
    elif proposed_endpoints_array.size > 0 and endpoint_array.size == 0:
        print("not correct--too many proposed endpoints but none ground truth endpoints exist")
        print("\nNEXT POINTS")
        continue
    #should skip for now if the endpoint array is size zero to avoid divide by zero error
    elif endpoint_array.size == 0:
        print("array size is 0")
        print("\nNEXT POINTS")
        continue
    else:                                                                                   
        for proposed_endpoint in proposed_endpoints_array:
            smallest_distance= find_euclidian_distance(proposed_endpoint, endpoint_array)
            print(smallest_distance)
            
            
            #compare to threshold and return yes or no 
            if smallest_distance < threshold:
                print("correct")
                num_correct = num_correct + 1
                print()
            else:
                print("wrong")
                print()
        #get precision and recall at end of each iteration 
        recall = get_recall(num_correct, endpoint_array)
        precision = get_precision(num_correct, proposed_endpoints_array)
        print("recall = " + str(recall))
        print("precision = " + str(precision))
        if recall > 100.0 or precision > 100.0:
            print("ERROR: RESULTS OVER 100% DETECTED")
        print()
        print("\nNEXT POINTS")
        
        
    

Seg id: 864691135909994000
Endpoint Array:
[[402584 228856  23991]
 [402985 229235  23554]]

Proposed Endpoint Array:
[[402567 228758  23991]
 [402990 229222  23552]]

99.46356116689167
correct

14.071247279470288
correct

recall = 100.0
precision = 100.0


NEXT POINTS
Seg id: 864691135247440303
Endpoint Array:
[[401612 224623  23991]
 [405257 226318  23620]
 [409162 219642  23891]]

Proposed Endpoint Array:
[[401467 224644  23990]
 [405021 226754  23620]
 [409143 219355  23868]]

146.5162107072115
correct

495.77414212522217
correct

288.54635676091976
correct

recall = 100.0
precision = 100.0


NEXT POINTS
Seg id: 864691134794123793
Endpoint Array:
[[401242 228382  24444]
 [400982 228457  24513]]

Proposed Endpoint Array:
[[400269 228563  24514]
 [400319 228912  24514]]

720.8370134780816
wrong

804.1113107026912
wrong

recall = 0.0
precision = 0.0


NEXT POINTS
Seg id: 864691134959738482
Endpoint Array:
[[401289 218721  23991]
 [399895 216533  23526]
 [406366 215719  23621]
 [403940

In [75]:
load.head()

,neuron,ng_link,seg_id,pink_pts,num_endpoints,Endpoints,Comments,proposed_endpoints
0,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"[(402584, 228856, 23991), (402985, 229235, 235...",NaN,"[(402567, 228758, 23991), (402990, 229222, 235..."
1,NaN,NaN,864691135247440303,"(401258, 224832, 24029)",3,"[(401612, 224623, 23991), (405257, 226318, 236...",NaN,"[(401467, 224644, 23990), (405021, 226754, 236..."
2,NaN,NaN,864691134794123793,"(401314, 228366, 24424)",2,"[(401242, 228382, 24444), (400982, 228457, 245...",NaN,"[(400269, 228563, 24514), (400319, 228912, 245..."
3,NaN,NaN,864691134959738482,"(400199, 220721, 24029)",7,"[(401289, 218721, 23991), (399895, 216533, 235...",NaN,"[(400448, 220975, 24009), (401240, 217916, 240..."
4,NaN,NaN,864691135314714227,"(397870, 232292, 24004)",2,"[(397867, 232230, 23999), (397854, 232252, 239...",NaN,"[(397815, 232393, 23996), (397931, 232013, 240..."


make threshold for each endpoint
experiment with box (cannot overlap)
for each endpoint see if it finds in box -- if yes true found endpont, if false --not an endpoint
find distance from predicted to ground truth 
could still do MSE 

Threshold: 
-find the size of the segment (maybe connected componenets)
-array from the skel
-flat is edge case 
-maybe a percentage 
--a hard number 





In [33]:
if len(proposed_endpoints_array.shape) == 0:
    print("correct")





correct


Need to complie results!!!! 

add to dataframe all of the precision and recallls 
plot the results and check for 
params don't have to be the best 


make a sepereate dataframe 

look: pd.concat

mention the params 

or have a collumn that has run number 

PRECISION: #test at ground truth / # test  

RECALL: # gt recovered / number ground truth 
(would be 100% if you found all the correct endponts...NO EXTRAS)